In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from tqdm.notebook import tqdm

In [4]:
fresh_query = """
SELECT from_npi, to_npi, patient_count
FROM hop
INNER JOIN [nppes]
ON from_npi = NPI
WHERE ProviderBusinessPracticeLocationAddressStateName = 'TN' AND HealthcareProviderPrimaryTaxonomySwitch = 'Y' AND ProviderBusinessPracticeLocationAddressCityName = 'NASHVILLE';
"""

with sqlite3.connect('NPPES_Data_Dissemination.sqlite') as db: 
    fresh_table = pd.read_sql(fresh_query, db)
    
fresh_table

,from_npi,to_npi,patient_count
0,1922001957,1295726032,81
1,1922001957,1578584199,129
2,1154324192,1295726032,110
3,1154324192,1578584199,202
4,1235132408,1093741464,112
...,...,...,...
135917,1104850130,1649228735,52
135918,1104850130,1700878360,85
135919,1104850130,1811955917,59
135920,1104850130,1861479545,217


In [5]:
fresh_table.to_csv('fresh_table.csv')

In [ ]:

facility_query = "SELECT NPI, [ProviderOrganizationName(LegalBusinessName)], ProviderBusinessPracticeLocationAddressPostalCode, HealthcareProviderTaxonomyCode FROM [nppes] WHERE EntityTypeCode = 2 AND ProviderBusinessPracticeLocationAddressCityName = 'NASHVILLE' AND ProviderBusinessPracticeLocationAddressStateName = 'TN' AND HealthcareProviderPrimaryTaxonomySwitch = 'Y' GROUP BY NPI, 'ProviderOrganizationName(LegalBusinessName)', ProviderBusinessPracticeLocationAddressPostalCode, HealthcareProviderTaxonomyCode"
    
facility_sqlite = pd.read_sql(facility_query, db)
    
facility_sqlite

In [ ]:
provider_query = "SELECT NPI, ProviderFirstName, [ProviderLastName(LegalName)], ProviderNamePrefixText, ProviderNameSuffixText, ProviderCredentialText, HealthcareProviderTaxonomyCode FROM [NPPES] WHERE EntityTypeCode = 1 AND ProviderBusinessPracticeLocationAddressCityName = 'NASHVILLE' AND ProviderBusinessPracticeLocationAddressStateName = 'TN' AND HealthcareProviderPrimaryTaxonomySwitch = 'Y' AND HealthcareProviderTaxonomyCode NOT NULL GROUP BY NPI, NPI, ProviderFirstName, [ProviderLastName(LegalName)], ProviderNamePrefixText, ProviderNameSuffixText, ProviderCredentialText, HealthcareProviderTaxonomyCode"

provider = pd.read_sql(provider_query, db)

provider

In [ ]:
taxonomy_nucc = pd.read_csv('nucc_taxonomy_230.csv', encoding= 'unicode_escape')

In [ ]:
provider = provider.merge(taxonomy_nucc, left_on = "HealthcareProviderTaxonomyCode", right_on = "Code")
provider

In [ ]:
provider['Classification'].unique()

In [ ]:
facility = facility_sqlite.merge(taxonomy_nucc, left_on = "HealthcareProviderTaxonomyCode", right_on = "Code")
facility = facility[facility['Grouping'] == 'Hospitals']
facility

In [ ]:
provider.to_sql('provider', db, if_exists = 'replace', index = False)

In [ ]:
facility.to_sql('facility', db, if_exists = 'replace', index = False)

In [ ]:
test_query = "SELECT Classification FROM provider"
provider_test = pd.read_sql(test_query, db)

provider_test

In [ ]:
specialist_query = "SELECT hop.from_npi, hop.to_npi, hop.patient_count, hop.transaction_count, ProviderFirstName AS First_Name, [ProviderLastName(LegalName)] AS Last_Name, [ProviderOrganizationName(LegalBusinessName)] AS Hospital_Name, provider.Classification FROM provider INNER JOIN hop ON provider.NPI = hop.from_npi LEFT JOIN facility ON hop.to_npi = facility.NPI ORDER BY hop.patient_count DESC;"




specialist_test = pd.read_sql(specialist_query, db)

specialist_test

In [ ]:
specialist_test['Hospital_Name'].unique()

In [ ]:
specialist_test['Classification'].unique()

In [ ]:
Nash_NPI = specialist_test[specialist_test.Hospital_Name.isin(['HCA HEALTH SERVICES OF TENNESSEE, INC.',
       'VANDERBILT UNIVERSITY MEDICAL CENTER',
       'HTI MEMORIAL HOSPITAL CORPORATION', 'SAINT THOMAS WEST HOSPITAL',
       'NASHVILLE GENERAL HOSPITAL', "BAPTIST WOMEN'S HEALTH CENTER LLC",
       'SELECT SPECIALTY HOSPITAL - NASHVILLE LLC', 'SETON CORPORATION',
       'CURAHEALTH NASHVILLE, LLC',
       'STATE OF TENNESSEE STATE F&A PAYROLL'])]

Nash_NPI

In [ ]:
Nash_NPI_Minus_Vandy = specialist_test[specialist_test.Hospital_Name.isin(['HCA HEALTH SERVICES OF TENNESSEE, INC.',
       'HTI MEMORIAL HOSPITAL CORPORATION', 'SAINT THOMAS WEST HOSPITAL',
       'NASHVILLE GENERAL HOSPITAL', "BAPTIST WOMEN'S HEALTH CENTER LLC",
       'SELECT SPECIALTY HOSPITAL - NASHVILLE LLC', 'SETON CORPORATION',
       'CURAHEALTH NASHVILLE, LLC',
       'STATE OF TENNESSEE STATE F&A PAYROLL'])]

Nash_NPI_Minus_Vandy

In [ ]:
Vandy_NPI = specialist_test[specialist_test.Hospital_Name.isin(['VANDERBILT UNIVERSITY MEDICAL CENTER'])]

Vandy_NPI

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
g = sns.FacetGrid(Nash_NPI_Minus_Vandy, col="Last_Name")
g.map(sns.barplot(
    data = Nash_NPI_Minus_Vandy[Nash_NPI_Minus_Vandy.Classification.isin(['Family Medicine', 'Orthopaedic Surgery'])],
    x = 'Hospital_Name',
    y = 'patient_count',
    hue = 'Classification',
    ax = ax
))


In [ ]:
Nash_Ortho_Fam = Nash_NPI[Nash_NPI.Classification.isin(['Orthopaedic Surgery', 'Family Medicine'])]

In [ ]:
sns.barplot(data=gdp_df.groupby(['Continent'])['Country'].nunique().to_frame().reset_index().rename(columns = {'Country' : 'Total_Countries'}), y='Continent', x='Total_Countries', hue='Continent', dodge=False); 

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

sns.barplot(
    data = Nash_NPI[Nash_NPI.Classification.isin(['Orthopaedic Surgery'])],
    x = 'Hospital_Name',
    y = 'patient_count',
    ax = ax
)
plt.xticks(rotation = 90);

In [ ]:
Nash_NPI[Nash_NPI.Classification.isin(['Family Medicine', 'Orthopaedic Surgery'])].groupby(['Hospital_Name'])['patient_count'].sum()

fig, ax = plt.subplots(figsize = (8,6))

sns.barplot(
    data = Nash_NPI[Nash_NPI.Classification.isin(['Family Medicine', 'Orthopaedic Surgery'])],
    x = 'Hospital_Name',
    y = y,
    hue = 'Classification',
    ax = ax
)
plt.xticks(rotation = 90);

In [ ]:
fig, ax = plt.subplots(figsize = (20,6))

sns.histplot(
    data = Nash_NPI[Nash_NPI.Classification.isin(['Family Medicine', 'Orthopaedic Surgery'])],
    x = 'Hospital_Name',
    y = 'patient_count',
    hue = 'Classification',
    ax = ax
)
plt.xticks(rotation = 90);

In [ ]:
Nash_Fam_Med = Nash_NPI[Nash_NPI.Classification.isin(['Orthopaedic Surgery'])]

Nash_Fam_Med['patient_count'].median()

In [ ]:
fig, ax = plt.subplots(figsize = (25,10))

sns.boxplot(
    data = Nash_NPI_Minus_Vandy[Nash_NPI_Minus_Vandy.Classification.isin(['Family Medicine', 'Orthopaedic Surgery'])],
    x = 'Last_Name',
    y = 'patient_count',
    hue = 'Classification',
    ax = ax
)
plt.xticks(rotation = 90);

In [ ]:
Nash_NPI_Minus_Vandy[Nash_NPI_Minus_Vandy['Last_Name'] == 'DEBOER']

In [ ]:
neo4j_output = pd.read_csv('neo4j_output.csv', encoding= 'unicode_escape')

In [ ]:
neo4j_output.to_sql('neo4j_output', db, if_exists = 'replace', index = False)

In [ ]:
neo4j_output.head()

In [ ]:
neo4j_output.groupby(['communityId'])['number'].count().sort_values(ascending = False).head(10)

In [ ]:
neo4j_output.groupby(['communityId'])['number'].count().sort_values(ascending = False).head(10).plot(kind = 'bar')


In [ ]:
specialist_group_id = "SELECT hop.from_npi, hop.to_npi, hop.patient_count, hop.transaction_count, ProviderFirstName AS First_Name, [ProviderLastName(LegalName)] AS Last_Name, [ProviderOrganizationName(LegalBusinessName)] AS Hospital_Name, provider.Classification, neo4j_output.communityId FROM provider INNER JOIN hop ON provider.NPI = hop.from_npi LEFT JOIN facility ON hop.to_npi = facility.NPI LEFT JOIN neo4j_output ON hop.from_npi = neo4j_output.number ORDER BY hop.patient_count DESC;"




specialist_group_id = pd.read_sql(specialist_group_id, db)

specialist_group_id

In [ ]:
specialist_group_id.groupby(['communityId', 'Classification'])['patient_count'].sum().sort_values(ascending = False).head(10).plot(kind = 'bar', color = 'orange')

In [ ]:
radiology_4384 = specialist_group_id.query('communityId == [4384] & Classification == "Internal Medicine"')

radiology_4384

In [ ]:
radiology_4384.groupby(['from_npi'])['patient_count'].sum().sort_values(ascending = False)

In [ ]:
specialist_group_id[specialist_group_id['from_npi'] == 1417131715].groupby(['from_npi'])['patient_count'].sum()

In [ ]:
specialist_group_id.query('communityId == [4413]').groupby(['from_npi'])['patient_count'].sum().sort_values(ascending = False)

In [ ]:
specialist_group_id.query('communityId == [4413] & from_npi == [1770519704]')[['Last_Name', 'First_Name']]

In [ ]:
#4454

specialist_group_id.query('communityId == [4454]').groupby(['from_npi'])['patient_count'].sum().sort_values(ascending = False)

In [ ]:
specialist_group_id.query('communityId == [4454] & from_npi == [1922091180]')

In [ ]:
#113
specialist_group_id.query('communityId == [113]').groupby(['from_npi'])['patient_count'].sum().sort_values(ascending = False)

In [ ]:
specialist_group_id.query('communityId == [113] & from_npi == [1093903544]')

In [ ]:
#4702

specialist_group_id.query('communityId == [4702]').groupby(['from_npi'])['patient_count'].sum().sort_values(ascending = False)

In [ ]:
specialist_group_id.query('communityId == [4702] & from_npi == [1306849765]')

In [ ]:
#I'm seeing that there are a lot of missing values, primarily because our decisions building the 'specialist_list' table that limits what we can determine from the exploration. I'm going to attempt to read in additional items to better fill out the 'Community ID' picture.

Community_ID = """

SELECT 




specialist_group_id = pd.read_sql(specialist_group_id, db)

specialist_group_id
